In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
os.listdir()

['.config', 'drive', 'sample_data']

In [3]:
#to change the current directory
os.chdir('/content/drive/My Drive')

In [4]:
#checing for the current working directory
pwd

'/content/drive/My Drive'

In [5]:
import sys, glob, subprocess
from itertools import product

import shutil
#from IPython.display import Image as JImage

import numpy as np
import cv2
import dlib
from PIL import Image
from skimage import io
from matplotlib.gridspec import GridSpec
from matplotlib import pyplot as plt

import tensorflow as tf

import keras
from keras.layers import Dense
from keras.models import Model
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras.applications.resnet50 import preprocess_input, decode_predictions
import warnings
warnings.filterwarnings("ignore")

In [6]:
#setting the batch size
batch_size=16

In [7]:
# ImageDataGenerator from Keras
datagen = ImageDataGenerator(
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest',
        #data_format="channels_last",
        preprocessing_function=preprocess_input,
)

In [8]:
#Defining the path of the training and test data 
train_dir,test_dir=('dataset/Train', 'dataset/Test')

In [9]:
#instead of loading all images into the ram we will do it in batch manner
# generator for train data
train_generator = datagen.flow_from_directory(
        train_dir,  
        target_size=(224, 224),  #all images will be resized to (224,224)
        batch_size=batch_size,
        class_mode='categorical',
        shuffle=True
)

Found 2204 images belonging to 2 classes.


In [11]:
# generator for validation data
validation_generator = datagen.flow_from_directory(
        test_dir, 
        target_size=(224, 224),  #all images will be resized to (224,224)
        batch_size=batch_size,
        class_mode='categorical',
        shuffle=True
)

Found 200 images belonging to 2 classes.


In [12]:
# will be using Resnet model pretrained on "imagenet" dataset
resnet50=keras.applications.resnet50.ResNet50(include_top=True,
                                     weights="imagenet", 
                                     input_tensor=None, 
                                     input_shape=None,
                                     pooling=None, 
                                     )

102973440/102967424 [==============================] - 5s 0us/step


In [20]:
#changing last two layers as we want only two output 
#keeping all layers trainable
output=Dense(2, activation="softmax",kernel_initializer=tf.keras.initializers.glorot_normal(seed=3))(resnet50.layers[-2].output) #new output layer
model=Model(resnet50.input, output) # new model with above output layer instead of 1000d from imagenet
model.compile(loss="categorical_crossentropy",optimizer='adam',metrics=['accuracy'])


In [26]:
#we will save the best model as best.h5 which has most validatin accuracy
checkpoint = tf.keras.callbacks.ModelCheckpoint("best.h5", monitor='val_accuracy', verbose=1, save_best_only=True,mode='max')
callbacks_list = [checkpoint]


In [27]:
model.fit(train_generator,
                          steps_per_epoch=len(train_generator),
                          epochs=25,
                          validation_data=validation_generator,
                          validation_steps=len(validation_generator),
                          callbacks=callbacks_list #to save beat model
) #train the model for 25 epochs

Epoch 1/25
138/138 [==============================] - ETA: 0s - loss: 0.0551 - accuracy: 0.9809
Epoch 00001: val_accuracy improved from -inf to 0.88000, saving model to best.h5
138/138 [==============================] - 57s 411ms/step - loss: 0.0551 - accuracy: 0.9809 - val_loss: 0.2734 - val_accuracy: 0.8800
Epoch 2/25
138/138 [==============================] - ETA: 0s - loss: 0.0732 - accuracy: 0.9741
Epoch 00002: val_accuracy improved from 0.88000 to 0.92500, saving model to best.h5
138/138 [==============================] - 59s 424ms/step - loss: 0.0732 - accuracy: 0.9741 - val_loss: 0.2260 - val_accuracy: 0.9250
Epoch 3/25
138/138 [==============================] - ETA: 0s - loss: 0.0617 - accuracy: 0.9782
Epoch 00003: val_accuracy improved from 0.92500 to 0.97500, saving model to best.h5
138/138 [==============================] - 59s 424ms/step - loss: 0.0617 - accuracy: 0.9782 - val_loss: 0.0722 - val_accuracy: 0.9750
Epoch 4/25
138/138 [==============================] - ETA: 0s